In [ ]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-2.5-flash", google_api_key="API-pT48g")
result = llm.invoke("song on salaar")
print(result)

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.agents import initialize_agent, Tool
from langchain.prompts import PromptTemplate

llm = GoogleGenerativeAI(model="gemini-2.5-flash", google_api_key="API-pT48g")

# Agent A: Deal Hunter
deal_hunter_prompt = PromptTemplate.from_template("""
You are Agent A, a shopping assistant.
User wants: {query}
Search results: {products}

Pick top 3 products that best fit the user's request based on value, price, and reviews.
Return in JSON: [{"name": "", "price": "", "rating": "", "link": "", "reason": ""}]
""")

# Agent B: Critic
critic_prompt = PromptTemplate.from_template("""
You are Agent B, a critical shopping analyst.
Agent A proposed: {recommendations}
Identify weaknesses, poor reviews, or issues.
Return feedback in JSON: [{"product": "", "criticism": ""}]
""")


In [12]:
products_data = """
[
  {
    "name": "HP 15s Ryzen 5 Laptop",
    "description": "15.6-inch display, 8GB RAM, 512GB SSD, ideal for ML workloads",
    "price": 58999,
    "reviews": "Great performance for students and data science tasks"
  },
  {
    "name": "ASUS Vivobook 14",
    "description": "Intel i5 12th Gen, 16GB RAM, 512GB SSD, lightweight design",
    "price": 64999,
    "reviews": "Smooth multitasking, slightly shorter battery life"
  },
  {
    "name": "Lenovo IdeaPad Slim 3",
    "description": "Ryzen 7, 16GB RAM, 1TB SSD, 15.6-inch Full HD display",
    "price": 67999,
    "reviews": "Excellent overall performance and battery backup"
  },
  {
    "name": "Dell Inspiron 14",
    "description": "Intel i7, 16GB RAM, 512GB SSD, sturdy build quality",
    "price": 69999,
    "reviews": "Great for coding, bit heavy to carry around"
  }
]
"""


In [ ]:
import json

products_data = """
[
  {"name": "HP 15s Ryzen 5 Laptop", "description": "15.6-inch display, 8GB RAM, 512GB SSD, ideal for ML workloads", "price": 58999, "reviews": "Great performance for students and data science tasks"},
  {"name": "ASUS Vivobook 14", "description": "Intel i5 12th Gen, 16GB RAM, 512GB SSD, lightweight design", "price": 64999, "reviews": "Smooth multitasking, slightly shorter battery life"},
  {"name": "Lenovo IdeaPad Slim 3", "description": "Ryzen 7, 16GB RAM, 1TB SSD, 15.6-inch Full HD display", "price": 67999, "reviews": "Excellent overall performance and battery backup"},
  {"name": "Dell Inspiron 14", "description": "Intel i7, 16GB RAM, 512GB SSD, sturdy build quality", "price": 69999, "reviews": "Great for coding, bit heavy to carry around"}
]
"""

def shopping_duo(query):
    products = json.loads(products_data)

    # 🧠 Agent A: The Deal Hunter
    a_prompt = f"""
    You are Agent A, a deal hunter.
    The user wants: {query}.
    Here are the available products: {products}.
    Pick the 2 best products based on price-to-performance ratio and value for money.
    
    Return your answer strictly in JSON format as:
    [
      {{ "name": "Product Name", "reason": "why you chose it" }},
      {{ "name": "Product Name", "reason": "why you chose it" }}
    ]
    """

    a_response = llm.invoke(a_prompt).strip()

    try:
        agent_a = json.loads(a_response)
    except json.JSONDecodeError:
        agent_a = [{"name": "Parsing Error", "reason": a_response}]

    # 🧠 Agent B: The Critic
    b_prompt = f"""
    You are Agent B, the critic.
    Agent A recommended: {agent_a}.
    Compare them critically based on performance, reviews, and battery life.
    Pick the final best one and explain why.

    Return your answer strictly in JSON format as:
    {{ "final_choice": "Product Name", "reason": "explanation" }}
    """

    b_response = llm.invoke(b_prompt).strip()

    try:
        agent_b = json.loads(b_response)
    except json.JSONDecodeError:
        agent_b = {"final_choice": "Parsing Error", "reason": b_response}

    return {
        "agent_a": agent_a,
        "agent_b": agent_b
    }



{
  "agent_a": [
    {
      "name": "Parsing Error",
      "reason": "```json\n[\n  {\n    \"name\": \"Lenovo IdeaPad Slim 3\",\n    \"reason\": \"This laptop offers the best price-to-performance ratio for machine learning under 70000. It features a powerful Ryzen 7 processor, ample 16GB RAM, and a generous 1TB SSD, which is crucial for storing large datasets and models, all at a highly competitive price.\"\n  },\n  {\n    \"name\": \"ASUS Vivobook 14\",\n    \"reason\": \"A strong second choice, offering excellent value for money. It comes with 16GB RAM and an Intel i5 12th Gen processor, providing solid performance for machine learning tasks at a more budget-friendly price point than other 16GB RAM options.\"\n  }\n]\n```"
    }
  ],
  "agent_b": {
    "final_choice": "Parsing Error",
    "reason": "```json\n{\n  \"final_choice\": \"Lenovo IdeaPad Slim 3\",\n  \"reason\": \"Based on the provided recommendations, the Lenovo IdeaPad Slim 3 emerges as the superior choice for machine le

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
import json

llm = GoogleGenerativeAI(model="gemini-2.5-flash", google_api_key="API-pT48g")

arbiter_prompt = PromptTemplate.from_template("""
Agent A's picks: {a}
Agent B's criticism: {b}

Now choose the final best product based on both sides' reasoning.
Explain your choice clearly.
""")

def get_final_recommendation(a, b):
    return llm.predict(arbiter_prompt.format(a=a, b=b))


In [17]:
query = "laptop under 70000 for machine learning"
results = shopping_duo(query)
print(results)


{'agent_a': [{'name': 'Parsing Error', 'reason': '```json\n[\n  {\n    "name": "Lenovo IdeaPad Slim 3",\n    "reason": "This laptop offers an excellent price-to-performance ratio for machine learning. It features a powerful Ryzen 7 processor, crucial 16GB of RAM, and a generous 1TB SSD, providing ample storage for datasets and models, all well within your budget."\n  },\n  {\n    "name": "ASUS Vivobook 14",\n    "reason": "A strong second choice, this laptop provides great value for money. It comes with a capable Intel i5 12th Gen processor and 16GB of RAM, which is essential for smooth machine learning workloads, at a very competitive price point."\n  }\n]\n```'}], 'agent_b': {'final_choice': 'Parsing Error', 'reason': '```json\n{\n  "final_choice": "Lenovo IdeaPad Slim 3",\n  "reason": "After a critical evaluation, the **Lenovo IdeaPad Slim 3** emerges as the superior choice for machine learning tasks. \\n\\n**Performance:** For machine learning, raw computational power, especially m

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
import json

# Initialize LLM
llm = GoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key="API-pT48g"
)

# Expanded and diversified product data
products_data = """
[
  {"name": "HP 15s Ryzen 5 Laptop", "description": "15.6-inch display, 8GB RAM, 512GB SSD, ideal for ML workloads", "price": 58999, "rating": 4.3, "reviews": "Great performance for students and data science tasks"},
  {"name": "ASUS Vivobook 14", "description": "Intel i5 12th Gen, 16GB RAM, 512GB SSD, lightweight design", "price": 64999, "rating": 4.2, "reviews": "Smooth multitasking, slightly shorter battery life"},
  {"name": "Lenovo IdeaPad Slim 3", "description": "Ryzen 7, 16GB RAM, 1TB SSD, 15.6-inch Full HD display", "price": 67999, "rating": 4.5, "reviews": "Excellent overall performance and battery backup"},
  {"name": "Dell Inspiron 14", "description": "Intel i7, 16GB RAM, 512GB SSD, sturdy build quality", "price": 69999, "rating": 4.4, "reviews": "Great for coding, bit heavy to carry around"},
  {"name": "Apple iPhone 15", "description": "A16 Bionic chip, 128GB, stunning camera and performance", "price": 79999, "rating": 4.7, "reviews": "Smooth experience and best-in-class camera"},
  {"name": "Samsung Galaxy S24", "description": "Dynamic AMOLED 2X, Snapdragon 8 Gen 3, 256GB", "price": 84999, "rating": 4.6, "reviews": "Excellent performance, great display, slightly pricey"},
  {"name": "OnePlus 12R", "description": "Snapdragon 8 Gen 2, 16GB RAM, 256GB storage, 5500mAh battery", "price": 49999, "rating": 4.4, "reviews": "Powerful phone for gamers and multitaskers"},
  {"name": "Sony WH-1000XM5 Headphones", "description": "Wireless Noise Cancelling, 30hr battery life, LDAC support", "price": 29999, "rating": 4.8, "reviews": "Best in class sound and ANC"},
  {"name": "Apple iPad Air (M2)", "description": "10.9-inch Liquid Retina display, M2 chip, 128GB", "price": 58999, "rating": 4.6, "reviews": "Perfect for students, artists, and light work"},
  {"name": "Logitech MX Master 3S Mouse", "description": "Wireless, ergonomic design, silent clicks", "price": 8999, "rating": 4.7, "reviews": "Superb comfort and productivity-focused"}
]
"""

def shopping_duo(query):
    products = json.loads(products_data)

    # Agent A – Deal Hunter
    a_prompt = f"""
    You are Agent A, a deal hunter.
    The user wants: {query}.
    Here are the available products: {products}.
    Pick the 3 best products based on overall value, performance, and rating.

    Return your answer strictly in JSON format as:
    [
      {{ "name": "Product Name", "reason": "why you chose it" }},
      {{ "name": "Product Name", "reason": "why you chose it" }},
      {{ "name": "Product Name", "reason": "why you chose it" }}
    ]
    """

    a_response = llm.invoke(a_prompt).strip()
    try:
        agent_a = json.loads(a_response)
    except json.JSONDecodeError:
        agent_a = [{"name": "Parsing Error", "reason": a_response}]

    # Agent B – Critic
    b_prompt = f"""
    You are Agent B, the critic.
    Agent A recommended: {agent_a}.
    Compare them critically based on price, reviews, and reliability.
    Choose the single best product and explain why in detail.

    Return your answer strictly in JSON format as:
    {{ "final_choice": "Product Name", "reason": "detailed explanation" }}
    """

    b_response = llm.invoke(b_prompt).strip()
    try:
        agent_b = json.loads(b_response)
    except json.JSONDecodeError:
        agent_b = {"final_choice": "Parsing Error", "reason": b_response}

    # 🧾 Final human-readable recommendation
    final_text = f"""
🛍️ **Top Picks (Agent A)**:
{chr(10).join([f"- {item['name']}: {item['reason']}" for item in agent_a])}

🤖 **Final Recommendation (Agent B)**:
✅ {agent_b.get('final_choice', 'N/A')} — {agent_b.get('reason', '')}
"""

    return final_text.strip()

# Example run
print(shopping_duo("best laptop or phone for AI/ML work under 70,000"))


🛍️ **Top Picks (Agent A)**:
- Parsing Error: ```json
[
  {
    "name": "Lenovo IdeaPad Slim 3",
    "reason": "This laptop offers the best overall package for AI/ML work under 70,000, featuring a powerful Ryzen 7 processor, ample 16GB RAM, and a generous 1TB SSD for data-intensive tasks. Its high rating and excellent performance make it a top choice."
  },
  {
    "name": "Dell Inspiron 14",
    "reason": "Equipped with a robust Intel i7 processor and 16GB RAM, the Dell Inspiron 14 provides excellent computational power for demanding AI/ML development and model training. Its sturdy build ensures reliability for continuous work."
  },
  {
    "name": "ASUS Vivobook 14",
    "reason": "A strong value pick, the ASUS Vivobook 14 combines a capable Intel i5 12th Gen processor with 16GB RAM, which is crucial for smooth multitasking and handling AI/ML projects efficiently. It's a great option for those seeking solid performance within budget."
  }
]
```

🤖 **Final Recommendation (Agent B)**:


In [ ]:
from langchain_google_genai import GoogleGenerativeAI
import json

# Initialize LLM for all agents
llm_main = GoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key="API_KEY"
)

# You could optionally use a different LLM for Agent C
llm_final_analyst = GoogleGenerativeAI(
    model="gemini-2.5",  # Can be different for diversity
    google_api_key="API-pT48g"
)

# Expanded and diversified product data
products_data = """
[
  {"name": "HP 15s Ryzen 5 Laptop", "description": "15.6-inch display, 8GB RAM, 512GB SSD, ideal for ML workloads", "price": 58999, "rating": 4.3, "reviews": "Great performance for students and data science tasks"},
  {"name": "ASUS Vivobook 14", "description": "Intel i5 12th Gen, 16GB RAM, 512GB SSD, lightweight design", "price": 64999, "rating": 4.2, "reviews": "Smooth multitasking, slightly shorter battery life"},
  {"name": "Lenovo IdeaPad Slim 3", "description": "Ryzen 7, 16GB RAM, 1TB SSD, 15.6-inch Full HD display", "price": 67999, "rating": 4.5, "reviews": "Excellent overall performance and battery backup"},
  {"name": "Dell Inspiron 14", "description": "Intel i7, 16GB RAM, 512GB SSD, sturdy build quality", "price": 69999, "rating": 4.4, "reviews": "Great for coding, bit heavy to carry around"},
  {"name": "Apple iPhone 15", "description": "A16 Bionic chip, 128GB, stunning camera and performance", "price": 79999, "rating": 4.7, "reviews": "Smooth experience and best-in-class camera"},
  {"name": "Samsung Galaxy S24", "description": "Dynamic AMOLED 2X, Snapdragon 8 Gen 3, 256GB", "price": 84999, "rating": 4.6, "reviews": "Excellent performance, great display, slightly pricey"},
  {"name": "OnePlus 12R", "description": "Snapdragon 8 Gen 2, 16GB RAM, 256GB storage, 5500mAh battery", "price": 49999, "rating": 4.4, "reviews": "Powerful phone for gamers and multitaskers"},
  {"name": "Sony WH-1000XM5 Headphones", "description": "Wireless Noise Cancelling, 30hr battery life, LDAC support", "price": 29999, "rating": 4.8, "reviews": "Best in class sound and ANC"},
  {"name": "Apple iPad Air (M2)", "description": "10.9-inch Liquid Retina display, M2 chip, 128GB", "price": 58999, "rating": 4.6, "reviews": "Perfect for students, artists, and light work"},
  {"name": "Logitech MX Master 3S Mouse", "description": "Wireless, ergonomic design, silent clicks", "price": 8999, "rating": 4.7, "reviews": "Superb comfort and productivity-focused"}
]
"""

def shopping_duo(query):
    products = json.loads(products_data)

    # --- Agent A: Deal Hunter ---
    a_prompt = f"""
    You are Agent A, a deal hunter.
    The user wants: {query}.
    Available products: {products}.
    Pick the 3 best products based on value, performance, and rating.

    Return strictly in JSON format:
    [
      {{ "name": "Product Name", "reason": "why you chose it" }},
      {{ "name": "Product Name", "reason": "why you chose it" }},
      {{ "name": "Product Name", "reason": "why you chose it" }}
    ]
    """
    a_response = llm_main.invoke(a_prompt).strip()
    try:
        agent_a = json.loads(a_response)
    except json.JSONDecodeError:
        agent_a = [{"name": "Parsing Error", "reason": a_response}]

    # --- Agent B: Critic ---
    b_prompt = f"""
    You are Agent B, the critic.
    Agent A recommended: {agent_a}.
    Critically compare them based on price, reviews, and reliability.
    Pick the single best product and explain why in detail.

    Return strictly in JSON format:
    {{ "final_choice": "Product Name", "reason": "detailed explanation" }}
    """
    b_response = llm_main.invoke(b_prompt).strip()
    try:
        agent_b = json.loads(b_response)
    except json.JSONDecodeError:
        agent_b = {"final_choice": "Parsing Error", "reason": b_response}

    # --- Agent C: Final Analyst ---
    c_prompt = f"""
    You are Agent C, the Final Analyst.
    You have:
    - Agent A's picks: {agent_a}
    - Agent B's evaluation: {agent_b}

    Your task:
    1. Review Agent A and B outputs carefully.
    2. Decide if Agent B's choice is optimal or choose another product if better.
    3. Provide a clear, final recommendation with reasoning.

    Output in human-readable text.
    """
    final_text = llm_final_analyst.invoke(c_prompt).strip()

    # --- Display results ---
    print("\n🛍️ Agent A (Deal Hunter) Picks:")
    for item in agent_a:
        print(f" - {item['name']}: {item['reason']}")

    print("\n🤖 Agent B (Critic) Final Pick:")
    print(f" ✅ {agent_b.get('final_choice', 'N/A')} — {agent_b.get('reason', '')}")

    print("\n🧩 Agent C (Final Analyst) Recommendation:")
    print(final_text)

    return {
        "agent_a": agent_a,
        "agent_b": agent_b,
        "agent_c": final_text
    }

# Example run
shopping_duo("best laptop or phone for AI/ML work under 70,000")
